# Pymatgen core functionality

## Structures, sites, and lattices

In [1]:
from pymatgen import Structure, Lattice

### Exercise: find your favorite oxide using the materials project rester.  Replace each oxygen atom with fluorine.

## SymmetryAnalyzer and StructureMatcher

## XRD, Bandstructure, and Density of States

## Phase and pourbaix diagrams

## Surfaces and Wulff Shapes

## Tensors